We will build Fr-En MT system --
Replace the paths wherever necessary --
Use the dataset uploaded in the Loop

In [ ]:
!pip3 install OpenNMT-py ## install OpenNMT

In [1]:
## check the current path and the contents
## upload the dataset sharded
!pwd   
#!ls

/home/stephen/Desktop/machine-translation-labs/lab4


In [2]:
# Unzip the data
!unzip en-fr.txt.zip


Archive:  en-fr.txt.zip
  inflating: README                  
  inflating: LICENSE                 
  inflating: TED2020.en-fr.en        
  inflating: TED2020.en-fr.fr        
  inflating: TED2020.en-fr.xml       


In [ ]:
##split the dataset into train dev and test  dev and test both containing 1000 sentence each
## use the train_test_split function



In [4]:
!pip3 install mosestokenizer ## tokenisation using Moses

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from mosestokenizer import *

In [ ]:
## Below we are performing tokenization for training set, perfrom it for validation and test set as well
## remember to replace the paths wherever applicable

from openfile import openfile
inputfile = openfile("train file name src")
outputfile = openfile("train file name tgt", "w")
tokenize = MosesTokenizer('en')
with inputfile,outputfile:
  for line in inputfile:
    print(*tokenize(line),file=outputfile)

In [ ]:
### Vocabulary configuration file

In [ ]:
config = '''# fr_en_build_vocab.yaml
## Where the samples will be written
save_data: path
## Where the vocab(s) will be written
src_vocab: path
tgt_vocab: path

# Corpus opts:
data:
    corpus_1:
        path_src:  train path
        path_tgt:  train path
    valid:
        path_src: valid path
        path_tgt:  valid path

src_seq_length: 150
tgt_seq_length: 150

# silently ignore empty lines in the data
skip_empty_level: silent
'''
with open("fr_en_build_vocab.yaml", "w") as config_yaml: # w+ means append use "w"
  config_yaml.write(config)

!cat fr_en_build_vocab.yaml

In [ ]:
config = '''# fr_en_training.yaml

## Where the samples will be written
# save_data: path
## Where the vocab(s) will be written
# src_vocab: src-vocab path
# tgt_vocab: tgt-vocab path
# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: train file path source
        path_tgt: train file target
    valid:
        path_src: valid-src
        path_tgt: valid tgt
world_size: 1
gpu_ranks: [0]

# Remove or modify these lines for bigger files
train_steps: 300000
valid_steps: 5000
learning_rate: 2

early_stopping: 5

src_vocab: path 
tgt_vocab: path

# Train on a single GPU
world_size: 1
gpu_ranks: [0]

# Where to save the checkpoints
save_model: path/model
save_checkpoint_steps: 5000
# train_steps: 300000
# valid_steps: 5000
'''
with open("en_fr_training.yaml", "w") as config_yaml:
  config_yaml.write(config)

!cat fr_en_training.yaml



In [ ]:
!onmt_build_vocab -config fr_en_build_vocab.yaml -n_sample -1

In [ ]:
!onmt_train -config fr_en_training.yaml

In [ ]:
!onmt_translate -model model_step_10000.pt -src test file -output opfile -gpu 0 -verbose ## replace model_step_10000.pt with your best checpoint

In [ ]:
!pip install sacrebleu
!pip install sacremoses

In [ ]:
### detokenise the output file and compute the Bleu score

In [ ]:
from sacrebleu.metrics import BLEU
import sacrebleu
from sacremoses import MosesDetokenizer
md = MosesDetokenizer(lang='lang')

refs = []

with open("path/test.fr") as test:
    for line in test: 
        line = line.strip().split() 
        line = md.detokenize(line) 
        refs.append(line)
    
#print("Reference 1st sentence:", refs[0])

refs = [refs]  # Yes, it is a list of list(s) as required by sacreBLEU


# Open the translation file by the NMT model and detokenize the predictions
preds = []

with open("output file") as pred:  
    for line in pred: 
        line = line.strip().split() 
        line = md.detokenize(line) 
        preds.append(line)  


# Calculate and print the BLEU score
bleu = sacrebleu.corpus_bleu(preds, refs)
print(bleu.score)